# Plot data overlaid on the map of Paris

In [6]:
%pylab inline
import json
import numpy as np
import seaborn as sns
sns.set_context('paper')
import pandas as pd
import matplotlib as mpl
from string import atof
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

# Download shapes of paris arrondissements
!wget -c https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements/75/communes.geojson -O data/75.geojson
with open('data/75.geojson') as f:
    data = json.load(f)

# Load datafile
df = pd.read_pickle("data/lbc_pandas.pkl")

# Useful definitions
def Gnormalize(x):
    """
        normalizes the 1D array/list x to be between 0. and 1.
    """
    x = sns.plt.np.asarray(x)
    x = x.copy()
    x-=x.min()
    x/=x.max()
    return x

from collections import defaultdict
def mean_outliers(x):
    _m = x.mean()
    _v = x.std()
    return x[(x>_m-_v) & (x<_m+_v)].mean()

def ddict(x,default=lambda : np.nan):
    """
        returns a default dict with content dict(x)
    """
    out = defaultdict(default)
    out.update(x)
    return out

tmp = []
for c in df[(df.Surface<50) & (df.Meuble_int==1)].groupby('Arrondissement'):
    tmp.append((c[0],mean_outliers(c[1].price_sqm)))

tmp = np.asarray(tmp)
aveprice_prenorm = ddict(tmp)

# normalize prices to compute colors per arrondissmement
tmp=tmp.T
tmp[1]=Gnormalize(tmp[1])

map_palette=sns.diverging_palette(220, 20, n=len(tmp),as_cmap=1)
_ = [(i,mpl.colors.rgb2hex(map_palette(j))) for i,j in tmp.T]
aveprice_color = ddict(_)

Populating the interactive namespace from numpy and matplotlib
--2017-01-24 09:37:29--  https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements/75/communes.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.



/home/astyonax/.anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Circle', 'f', 'Rectangle', 'Polygon']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Heatmap on paris based average price per arrondissement
We will assign a colorcode to each arrondissmement, based on the **average price**

In [7]:
for j,feature in enumerate(data['features']):
    where = atof(feature['properties']['code'])-75100
    price = aveprice_prenorm[where]
    if not price:
        continue
#     price = flats_per_arrondissement[where]
    color = aveprice_color[where]
    
    feature['properties']['style'] = {'color':color, 'weight': .5, 'fillColor':color, 'fillOpacity':0.75}
    feature['properties']['district']='Arrondissement: {0:.0f}, price: {1:.0f}eu/sqm'.format(where,price)

layer = GeoJSON(data=data, hover_style={'fillOpacity': 1})

def hover_handler(event=None, id=None, properties=None):
    sys.stdout.write("\r" + properties['district'])
    sys.stdout.flush()
    
center = [48.85906816414895, 2.343006134033203]
zoom = 13
m = Map(center=center, zoom=zoom)
layer.on_hover(hover_handler)
m.add_layer(layer)
m.height='500px'
print 'price per sqm (<50smq, meuble\') per district'
print 'You need to run the notebook to see the map :/' 
m

price per sqm (<50smq, meuble') per district
You need to run the notebook to see the map :/
Arrondissement: 12, price: 31eu/sqm

In [8]:
from IPython.display import HTML
import urllib2

style=open('style.css','r').read()
HTML("""
<style>{0}</style>
""".format(style))